# Лабораторная 2

In [19]:
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [117]:
df = pd.read_csv("data\heart.csv")

In [118]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [119]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X, tree):
        '''Предсказать данные'''

In [120]:
class DecisionTree(Classificator):
    
    def train(self, X, Y, tabs="", depth = 0, max_depth=None, leafValue=None):
        entropy = self.calculate_entropy(Y)
        node = {}
        if entropy == 0 or depth == max_depth or len(X) == leafValue or len(X.columns) == 0:
            p = 0
            if len(Y.loc[Y != 1]) == 0:
                p = 1
            elif len(Y.loc[Y == 1]) == 0:
                p = 0
            else:
                p = len(Y.loc[Y == 1]) / len(Y)
            node['label'] = p
            return node
        igs = []
        for index, row in X.iterrows():
            inner_igs = []
            for column in X:
                predicate = row[column]
                left = X[column].loc[X[column] <= predicate]
                right = X[column].loc[X[column] > predicate]
                Y_left = Y.loc[left.index]
                Y_right = Y.loc[right.index]
                groups = [[len(left), self.calculate_entropy(Y_left)], [len(right), self.calculate_entropy(Y_right)]]
                ig = self.information_gain(Y, entropy, groups)
                inner_igs.append(Predicate(column, ig, predicate))
            igs.append(self.get_max_ig(inner_igs))

        predicate = self.get_max_ig(igs)
        
        
        X_left = X.loc[X[predicate.attribute] <= predicate.predicate].drop(predicate.attribute, axis=1)
        X_right = X.loc[X[predicate.attribute] > predicate.predicate].drop(predicate.attribute, axis=1)
        Y_left = Y.loc[X_left.index]
        Y_right = Y.loc[X_right.index]
        depth += 1
        node['attribute'] = predicate.attribute
        node['value'] = predicate.predicate
        node['nodes'] = {}
        node['nodes']['<='] = DecisionTree().train(X_left, Y_left, depth=depth, max_depth=max_depth)
        node['nodes']['>'] = DecisionTree().train(X_right, Y_right, depth=depth, max_depth=max_depth)
        return node
    
    def predict(self, X, tree):
        X['target'] = [0 for i in range(len(X))]
        return self.set_label(X, tree)
    
    def set_label(self, X, tree):
        if 'label' in tree:
            X.target = tree['label']
            return X
        left = X.loc[X[tree['attribute']] <= tree['value']]
        X.loc[X[tree['attribute']] <= tree['value']] = self.set_label(left, tree['nodes']['<='])
        right = X.loc[X[tree['attribute']] > tree['value']]
        X.loc[X[tree['attribute']] > tree['value']] = self.set_label(right, tree['nodes']['>'])
        return X
    
    def calculate_entropy(self, Y):
        count = len(Y)
        entropy = 0
        for count_class in Y.value_counts():
            p = count_class / count
            entropy -= p * np.log2(p)
        return entropy
    
    
    def information_gain(self, Y, entropy0, groups):
        N = len(Y)
        sum_entropies = 0
        for group in groups:
            group
            Ni = group[0]
            Si = group[1]
            sum_entropies += Ni / N * Si
        return entropy0 - sum_entropies
    
    def get_max_ig(self, igs):
        information_gain = igs[0]
        for ig in igs:
            if ig.ig > information_gain.ig:
                information_gain = ig
        return information_gain

In [121]:
class RandomForest(Classificator):
    tree = DecisionTree()
    model = []
    
    def train(self, X, Y, N=10, max_depth=None):
        for i in range(N):
            Xn = X.sample(int(len(X) * 2 / 3), replace=True)
            Yn = Y.loc[Xn.index]
            self.model.append(self.create_tree(Xn, Yn, max_depth=max_depth))
        return self.model
    
    def create_tree(self, X, Y, depth=0, max_depth=None, min_leaf=None):
        entropy = self.tree.calculate_entropy(Y)
        node = {}
        if entropy == 0 or len(X.columns) == 0:
            p = 0
            if len(Y.loc[Y != 1]) == 0:
                p = 1
            elif len(Y.loc[Y == 1]) == 0:
                p = 0
            else:
                p = len(Y.loc[Y == 1]) / len(Y)
            node['label'] = p
            return node
        
        X = X.sample(int(np.sqrt(len(X.columns))), axis=1)
        igs = []
        for index, row in X.iterrows():
            inner_igs = []
            for column in X:
                predicate = row[column]
                left = X[column].loc[X[column] <= predicate]
                right = X[column].loc[X[column] > predicate]
                Y_left = Y.loc[left.index]
                Y_right = Y.loc[right.index]
                groups = [[len(left), self.tree.calculate_entropy(Y_left)], [len(right), self.tree.calculate_entropy(Y_right)]]
                ig = self.tree.information_gain(Y, entropy, groups)
                inner_igs.append(Predicate(column, ig, predicate))
            igs.append(self.tree.get_max_ig(inner_igs))

        predicate = self.tree.get_max_ig(igs)
        
        
        X_left = X.loc[X[predicate.attribute] <= predicate.predicate].drop(predicate.attribute, axis=1)
        X_right = X.loc[X[predicate.attribute] > predicate.predicate].drop(predicate.attribute, axis=1)
        Y_left = Y.loc[X_left.index]
        Y_right = Y.loc[X_right.index]
        depth += 1
        node['attribute'] = predicate.attribute
        node['value'] = predicate.predicate
        node['nodes'] = {}
        node['nodes']['<='] = self.create_tree(X_left, Y_left, depth=depth, max_depth=max_depth)
        node['nodes']['>'] = self.create_tree(X_right, Y_right, depth=depth, max_depth=max_depth)
        return node
    
    def predict(self, X):
        prediction_list = []
        res = []
        for m in self.model:
            prediction_list.append(self.tree.predict(X, tree=m))
        for i in prediction_list[0].target.index:
            label = 0
            for t in prediction_list:
                label += t.target[i]
            label /= len(prediction_list)
            res.append(label)
        return res

In [122]:
class GradientBoosting(Classificator):
    model = None
    
    def train(X, Y, M, L=None, h=None):
        for t in M:
            for i in n:
                r = pseudo(y, f(x))
            h = DecisionTree(x, r)
            ro = min(sum(L(y), f(x) + ro * h(x)))
        return
    
    def predict(self, X):
        pass

In [123]:
def read_tree(t, tabs='', d=''):
    print(tabs+'predicate: ' + d)
    if 'label' in t:
        print(tabs+'label: ' + str(t['label']))
    else:
        print(tabs+'attr: ' + t['attribute'])
        print(tabs+'val: ' + str(t['value']))
        read_tree(t['nodes']['<='], tabs+'\t', d='<=')
        read_tree(t['nodes']['>'], tabs+'\t', d='>')
    return

In [124]:
class Predicate:
    attribute = ""
    ig = 0
    predicate = 0
    
    def __init__(self, attribute, ig, predicate):
        self.attribute = attribute
        self.ig = ig
        self.predicate = predicate

In [ ]:
X = df[df.columns[:-1]][:10000]
y = df.Class[:10000]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree()
t = tree.train(X_train, Y_train, max_depth=2)
predicted = tree.predict(X_test, t)

In [125]:
X = df[df.columns[:-1]]
y = df.target

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

forest = RandomForest()
f = forest.train(X_train, Y_train, max_depth=1)

In [126]:
predicted = forest.predict(X_test)

C:\Users\adugeen\anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\adugeen\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\adugeen\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [128]:
pred = list(map(lambda x : 0 if x < 0.5 else 1, predicted))

In [131]:
i = 0
cnt = 0
for t in Y_test:
    if t != pred[i]:
        cnt += 1
    i += 1
cnt / len(Y_test)

0.29508196721311475

In [127]:
predicted

[0.16528925619834708,
 0.924170616113744,
 0.16528925619834708,
 0.16528925619834708,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.0,
 0.0,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.47058823529411764,
 0.16528925619834708,
 0.16528925619834708,
 0.0,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.16528925619834708,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.16528925619834708,
 0.924170616113744,
 0.16528925619834708,
 0.16528925619834708,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.16528925619834708,
 0.924170616113744,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.924170616113744,
 0.16528925619834708,
 0.924170616113744,
 0.924170616113744,
 0.0,
 0.16528925619834708,
 0.165289256198347